In [47]:
import pandas as pd
import numpy as np
import sqlite3 as db

In [2]:
%pwd

'/home/campus-13/Documents/Projet_BDD'

In [54]:
#Import des fichiers --> df
routes_df=pd.read_csv('./SEM-GTFS/routes.txt')
# stops_df=pd.read_csv('./SEM-GTFS/stops.txt')
# stop_times_df=pd.read_csv('./SEM-GTFS/stop_times.txt')
# agency_df=pd.read_csv('./SEM-GTFS/agency.txt')
# trips_df=pd.read_csv('./SEM-GTFS/trips.txt')

In [37]:
routes_df.shape

(50, 7)

In [38]:
#Transformation des fichiers en dictionnaires
routes_dic=routes_df.to_dict(orient='dict')
stops_dic=stops_df.to_dict(orient='dict')
stop_times_dic=stop_times_df.to_dict(orient='dict')
agency_dic=agency_df.to_dict(orient='dict')
trips_dic=trips_df.to_dict(orient='dict')

In [39]:
# Fonction transformation données mais pas création de fichier sql
def gen_insert_query(table_name:str, a_dict:dict) -> str:
    #table_name= stops_insert.sql par exple
    #df --> attention à gérer les NaN
    # https://sql.sh/cours/insert-into
    # INSERT INTO table-name (column-names)  VALUES (values) 

    insert_query_tpl = "INSERT INTO {} {} VALUES {} "
    keys, values = zip(*a_dict.items())
    
    compiled_query = insert_query_tpl.format(table_name, keys, values)
    
    return compiled_query

In [40]:
# gen_insert_query(routes_df,routes_dic)

In [41]:
#Fonction pour mettre en forme .txt puis création fichier sql
def gen_insert_file(filename,tablename,df):
    with open(filename,'w') as file:
        file.write("BEGIN;") #BEGIN et COMMIT sont à ajouter pour optimiser le process/plus rapide
        for i in range(len(df)):
            line=df.iloc[i].dropna()
            query= "INSERT INTO {} {} VALUES {};\n ".format(tablename, tuple(line.index), tuple(line.to_list()))
            file.write(query)
        file.write("COMMIT;")
    return
#Pour import dans DB --> terminal : sqlite3 gtfs_tag.db < routes.sql

In [42]:
gen_insert_file("./routes.sql","routes", routes_df)

In [44]:
# import pandas
# import sqlite3 as db

# DB = db.connect('DB.db')
# csv1 = pandas.read_csv('C:\\…..csv')
# csv1.to_sql(name='Orders', con=DB, if_exists='replace')
# #replace is one of three options available for the if_exists parameter
# DB.close()

In [57]:
#Création du connecteur
DB=db.connect('gtfs_tag.db')
connecteur=sqlite3.connect('gtfs_tag.db')


routes_df.to_sql(name='routes',con=connecteur, if_exists='replace') #if_exists='append' quand on veut rajouter

#Transformations et chargements des fichiers concernés
stops_df=pd.read_csv('./SEM-GTFS/stops.txt') #ajouter na_values?
stops_df.to_sql(name='stops',con=connecteur, if_exists='replace')

stop_times_df=pd.read_csv('./SEM-GTFS/stop_times.txt')
stop_times_df.to_sql(name='stops_times',con=connecteur, if_exists='replace')

agency_df=pd.read_csv('./SEM-GTFS/agency.txt')
agency_df.to_sql(name='agency',con=connecteur, if_exists='replace')

trips_df=pd.read_csv('./SEM-GTFS/trips.txt')
trips_df.to_sql(name='trips',con=connecteur, if_exists='replace')

#Fermeture de la connection
DB.close()
                
                


## Plan A:

- Formaliser le schema.sql à importer dans USer interface for sql

- Utiliser pandas pour créer la DB:

    a - DB = db.connect("name.db")

    b - connecteur=sqlite3.connect("name.db")
    
    c - importer ton fichier txt et le transformer en dataframe --> pd.read_csv("fichier.txt")
    
    d - transformer ton df en fichier sql -->        
        namedf.to_sql(name='nomdelatable',con=connecteur,if_exists='replace ou append')
        # append will allow to keys to be preserved like defined in the schema
        #index=False if you do not want extra-index to be added automatically
    
    e - DB.close()
    
    
## Plan B:

- Formaliser le schema.sql à importer dans USer interface for sql

- Recréer la fonction permettant d'implémenter le sql

- Ecriture dans DB : dans terminal --> sqlite3 fichier.db < fichier.sql 

